In [5]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

!pip install ipython-autotime
%load_ext autotime

time: 280 µs (started: 2022-03-23 18:54:33 +00:00)


### Download & Process Glove. Ignore if mod file exists

In [6]:
#!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
#!unzip glove*.zip

model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt')
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False)

def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

generate_norm_embedding(model_glove,'glove_norm_300.mod')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1917494 [00:00<?, ?it/s]

time: 7min 9s (started: 2022-03-23 18:54:45 +00:00)


### Load processed glove

In [7]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True) # ss


time: 33.5 s (started: 2022-03-23 19:02:57 +00:00)


In [8]:
current_model = model_gn 


time: 1.26 ms (started: 2022-03-23 19:03:30 +00:00)


### Load and process antonyms. Ignore if final_antonym_list exists.

In [ ]:
list_antonym = []

with open('Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))

with open('Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

### Load final_antonym_list as antonym list

In [9]:
import pandas as pd

list_antonym = pd.read_pickle(r'final_antonym_list')

time: 6.93 ms (started: 2022-03-23 19:03:30 +00:00)


### Decide on the size of the antonym vector


In [18]:
num_antonym = 1468

## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)
time: 27.6 ms (started: 2022-03-23 19:07:27 +00:00)


### Subset Dimension Selection Method


In [16]:
import random
import scipy

random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = scipy.spatial.distance.cdist(np.array(antonymy_vector),np.array(antonymy_vector),'cosine')
dimension_similarity_matrix = abs(1-dimension_similarity_matrix)
        
def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
    return final_position_index

time: 665 ms (started: 2022-03-23 19:06:06 +00:00)


## Select POLAR dimensions

#### Generate the RANDOM DIMENSION Order


In [14]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

1468
time: 5.37 ms (started: 2022-03-23 19:05:27 +00:00)


#### Generate the ORTHOGONAL DIMENSION Order


In [19]:
orthogonal_antonymy_vector =np.array(select_subset_dimension(antonymy_vector, num_antonym))  
print(orthogonal_antonymy_vector.shape)

working list is ready, shape (1468, 300)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1467 [00:00<?, ?it/s]

(1468,)
time: 3min 18s (started: 2022-03-23 19:07:31 +00:00)


#### Generate the MAXIMUM VARIANCE DIMENSION Order


In [23]:
import torch

embedding_size = antonymy_vector.shape[0]   
print('The embedding size is', embedding_size)


variance_antonymy_vector_inverse = np.linalg.pinv(np.transpose(antonymy_vector))
variance_antonymy_vector_inverse = torch.tensor(variance_antonymy_vector_inverse)

embedding_matrix = []

current_model_tensor = torch.t(torch.tensor(current_model.wv.vectors))

The embedding size is 1468
time: 6.43 s (started: 2022-03-23 19:19:13 +00:00)


In [41]:
torch.var_mean(torch.matmul(, current_model_tensor[::,:100]), axis = 1)

torch.Size([1468])

time: 9.5 ms (started: 2022-03-23 19:28:13 +00:00)


In [60]:
var_list = [None for x in range(20)] # variance for each antonym in each batch

for i in range(19):  # the first 19 batches, each of size 100k
  temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,100000*i:100000*i+100000])
  temp_var_mean = torch.var(temp, axis = 1)
  var_list[i] = temp_var_mean[0].numpy()
  del temp

temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,1900000:])
temp_var_mean = torch.var(temp, axis = 1)
var_list[19] = temp_var_mean[0].numpy()
del temp

IndexError: ignored

time: 49.6 s (started: 2022-03-23 19:42:31 +00:00)


In [71]:
# using lazy approach. assume each batch is independent and the overall variance is the average variance over all batches

variance_list = np.mean(np.array(var_list),axis = 0)

variance_antonymy_vector = [each for each in sorted(range(len(variance_list)), key=lambda i: variance_list[i], reverse=True)]


time: 6.31 ms (started: 2022-03-23 19:46:35 +00:00)


### Begin POLAR embedding